#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# get total number of rows 
print(len(full_data_rows_list))

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


#### Creating a Cluster

In [5]:
#  make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
try:
    session.execute("""
    create keyspace if not exists project
    with replication =
    {'class':'SimpleStrategy', 'replication_factor':1}""")
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Set KEYSPACE project
try:
    session.set_keyspace('project')
except Exception as e:
    print(e)


In [14]:
## Create a table to do query by sessionId and itemInSession

create_query = "create table if not exists song_by_session_item (sessionId int, itemInSession int ,song text, artist text,length float, primary key(sessionId, itemInSession))"       
try:
    session.execute(create_query)
except Exception as e:
    print(e)

In [16]:

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
##  Insert each line into table
        query = "insert into song_by_session_item (sessionId, itemInSession, song, artist, length) values (%s,%s,%s,%s,%s)"
        
        try:
            session.execute(query, (int(line[8]), int(line[3]), line[9], line[0], float(line[5])))
        except Exception as e:
            print(e)
            
        

In [18]:

query1 = "select artist, song, length from song_by_session_item where sessionId = 338 and itemInSession = 4;"
try:
    rows = session.execute(query1)
except Exception as e:
    print(e)
    
for row in rows:
    print(row)

## The output is 
 ## Row(artist='Faithless', song='Music Matters (Mark Knight Dub)', length='495.3073')

Row(artist='Faithless', song='Music Matters (Mark Knight Dub)', length=495.30731201171875)


In [19]:
## Create the 2nd table to enable the query by session id and user id
create_query2 = "create table if not exists song_by_user_session (userId int, sessionId int, itemInSession int, artist text, firstName text, lastName text, song text, primary key((userId, sessionId),itemInSession))"       
try:
    session.execute(create_query2)
except Exception as e:
    print(e)
    
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Insert each line into table
        query = "insert into song_by_user_session (userId, sessionId,itemInSession, artist, firstName, lastName, song) values (%s,%s,%s,%s,%s,%s,%s)"
        
        try:
            session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[1], line[4], line[9]))
        except Exception as e:
            print(e)   
            
query2 = "select artist, firstName,lastName, song from song_by_user_session where sessionId=182 and userId=10;"
try:
    rows = session.execute(query2)
except Exception as e:
    print(e)
    
for row in rows:
    print(row)

## The output is 
 ## Row(artist='Down To The Bone', firstname='Sylvie', lastname='Cruz', song="Keep On Keepin' On")
 ## Row(artist='Three Drives', firstname='Sylvie', lastname='Cruz', song='Greece 2000')
 ## Row(artist='Sebastien Tellier', firstname='Sylvie', lastname='Cruz', song='Kilometer')
 ## Row(artist='Lonnie Gordon', firstname='Sylvie', lastname='Cruz', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)')

Row(artist='Down To The Bone', firstname='Sylvie', lastname='Cruz', song="Keep On Keepin' On")
Row(artist='Three Drives', firstname='Sylvie', lastname='Cruz', song='Greece 2000')
Row(artist='Sebastien Tellier', firstname='Sylvie', lastname='Cruz', song='Kilometer')
Row(artist='Lonnie Gordon', firstname='Sylvie', lastname='Cruz', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)')


In [20]:
## create the 3rd table to enable the query by song and user id
create_query3 = "create table if not exists user_by_song (song text, userId int,  firstName text,  lastName text, primary key(song, userId))"       
try:
    session.execute(create_query3)
except Exception as e:
    print(e)
    
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Insert each line into table
        query3 = "insert into user_by_song (song, userId, firstName, lastName) values (%s,%s,%s,%s)"
      
        try:
            session.execute(query3, (line[9], int(line[10]),line[1], line[4]))
        except Exception as e:
            print(e)   

query3 = "select firstName, lastName from user_by_song where song='All Hands Against His Own';"
                    
try:
    rows = session.execute(query3)
except Exception as e:
    print(e)
    
for row in rows:
    print(row)

## The output is:   
 ## Row(firstname='Sara', lastname='Johnson')
 ## Row(firstname='Jacqueline', lastname='Lynch')
 ## Row(firstname='Tegan', lastname='Levine')

Row(firstname='Jacqueline', lastname='Lynch')
Row(firstname='Tegan', lastname='Levine')
Row(firstname='Sara', lastname='Johnson')


In [4]:
##  Drop the table before closing out the sessions

In [13]:
query_drop = "drop table song_by_session_item"
try:
    rows = session.execute(query_drop)
except Exception as e:
    print(e)

query_drop2 = "drop table song_by_user_session"
try:
    rows = session.execute(query_drop2)
except Exception as e:
    print(e)

query_drop3 = "drop table user_by_song"
try:
    rows = session.execute(query_drop3)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [24]:
session.shutdown()
cluster.shutdown()